In [2]:
# !pip install peft trl bitsandbytes accelerate evaluate rouge_score bert_score

In [1]:
# !pip install -U datasets

In [3]:
import json
import re
import wandb
import os
import evaluate
import pandas as pd
import torch
import os
import gc
import numpy as np

from pprint import pprint
from kaggle_secrets import UserSecretsClient
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login, login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
from warnings import filterwarnings

filterwarnings('ignore')

rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')

2024-03-15 13:26:18.486509: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 13:26:18.486561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 13:26:18.488039: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import wandb
# Start a W&B run
wandb.login()
wandb.init(project="dialog-summarization")

wandb: Currently logged in as: linusx. Use `wandb login --relogin` to force relogin


In [ ]:
os.environ['TOKENIZERS_PARALLELISM'] = 'True'

In [5]:
df = pd.read_csv("/kaggle/input/dialogsum/CSV/train.csv", nrows=2000)
df.columns = [str(q).strip() for q in df.columns]

dataset = Dataset.from_pandas(df)

In [6]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()

def generate_training_prompt(
    conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""    
### Instruction: {system_prompt}
        
### Input: {conversation.strip()}
               
### Response: {summary}
""".strip()

In [7]:
def create_conversation_text(data_point):
    return data_point["dialogue"]


def generate_text(data_point):
    summary = data_point["summary"]
    conversation_text = create_conversation_text(data_point)
    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": generate_training_prompt(conversation_text, summary),
    }

# Example usage with a new dataset format
example_data_point = {
    "id": "train_0",
    "dialogue": "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today? #Person2#: I found it would...",
    "summary": "Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll gi...",
    "topic": "get a check-up"
}


example = generate_text(example_data_point)
print(example["text"])

### Instruction: Below is a conversation between a human and an AI agent. Write a summary of the conversation.
        
### Input: #Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today? #Person2#: I found it would...
               
### Response: Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll gi...


In [8]:
from datasets import Dataset

def process_dataset(data: Dataset) -> Dataset:
    """
    This function processes the dataset to include only the necessary columns.
    """
    # First, apply generate_text to each record in the dataset
    processed_data = data.map(generate_text)


    # Then, remove unnecessary columns
    columns_to_remove = [col for col in processed_data.column_names if col not in ["conversation", "summary", "text"]]
    return processed_data.remove_columns(columns_to_remove)

In [9]:
# Process the entire dataset
processed_dataset = process_dataset(dataset)


# Split the processed dataset into train, validation, and test sets
train_dataset = processed_dataset.shuffle(seed=42).select(range(0, int(0.8 * len(processed_dataset))))
validation_dataset = processed_dataset.shuffle(seed=42).select(range(int(0.8 * len(processed_dataset)), int(0.9 * len(processed_dataset))))
test_dataset = processed_dataset.shuffle(seed=42).select(range(int(0.9 * len(processed_dataset)), len(processed_dataset)))

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [10]:
CUSTOM_DEVICE_MAP =  {
 'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 0,
 'model.layers.13': 0,
 'model.layers.14': 1,
 'model.layers.15': 1,
 'model.layers.16': 1,
 'model.layers.17': 1,
 'model.layers.18': 1,
 'model.layers.19': 1,
 'model.layers.20': 1,
 'model.layers.21': 1,
 'model.layers.22': 1,
 'model.layers.23': 1,
 'model.layers.24': 1,
 'model.layers.25': 1,
 'model.layers.26': 1,
 'model.layers.27': 1,
 'model.layers.28': 1,
 'model.layers.29': 1,
 'model.layers.30': 1,
 'model.layers.31': 1,
 'model.norm': 1,
 'lm_head': 1
}

In [11]:
MODEL_NAME = 'NousResearch/Llama-2-7b-chat-hf'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    use_safetensors=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map=CUSTOM_DEVICE_MAP,
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
model.config.use_cache = False
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 '_load_in_8bit': False,
 '_load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16',
 'load_in_4bit': True,
 'load_in_8bit': False}

In [15]:
lora_alpha = 32
lora_dropout = 0.05
lora_r = 16

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [17]:
def generate_prompt(
    conversation: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
""".strip()


def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to('cuda:0')
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)


def generate_summaries(model, dataset, tokenizer, num_samples=5):
    summaries = []
    for i, example in enumerate(dataset):
        if i >= num_samples:
            break
        print(i)
        prompt = generate_prompt(example['conversation'])
        summary = summarize(model, prompt)
        summaries.append({'conversation': example['conversation'], 'generated_summary': summary})
    return summaries

In [1]:
# Generate summaries before fine-tuning
original_summaries = generate_summaries(model, test_dataset, tokenizer, num_samples=50)

# Convert to DataFrame and log to W&B
df_original = pd.DataFrame(original_summaries)
actual_summaries = pd.DataFrame(test_dataset)

#### Compute scores before finetuning

In [ ]:
rouge_scores = rouge.compute(predictions=df_original.generated_summary, references=actual_summaries.summary.iloc[:50])
bert_scores = bert_score.compute(predictions=df_original.generated_summary, references=actual_summaries.summary.iloc[:50], lang="en")

print('Rouge: ', rouge_scores)
print('BERTScore Precision: ', np.mean(bert_scores['precision']))
print('BERTScore Recall: ', np.mean(bert_scores['recall']))
print('BERTScore F1: ', np.mean(bert_scores['f1']))

In [19]:
OUTPUT_DIR = "dialog-summarization-llama-2-finetuned"

training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=20,
    evaluation_strategy="epoch",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="wandb",  # Set report_to here
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [20]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [21]:
# Fine-tune your model
trainer.train()

# Generate summaries after fine-tuning
fine_tuned_summaries = generate_summaries(trainer.model, test_dataset, tokenizer, num_samples=5)

# Convert to DataFrame and log to W&B
df_fine_tuned = pd.DataFrame(fine_tuned_summaries)
wandb.log({"fine_tuned_summaries": wandb.Table(dataframe=df_fine_tuned)})

Epoch,Training Loss,Validation Loss
1,1.745500,1.654858
2,1.326700,1.320741
3,1.155600,1.229804
4,1.090800,1.223995
5,1.121900,1.221663
6,1.094100,1.226037
7,1.159700,1.235341
8,1.073700,1.244705
9,1.022800,1.252475
10,1.040100,1.271391


0
1
2
3
4


In [24]:
trainer.save_model('/kaggle/working/best_ckpt')

In [2]:
import shutil
shutil.make_archive('llama2_dialog_sum_2', 'zip', '/kaggle/working/best_ckpt')

#### Load model and score it

In [ ]:
MODEL_NAME = '/kaggle/working/best_ckpt'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    use_safetensors=True,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map=CUSTOM_DEVICE_MAP,
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
finetuned_summaries_df = pd.DataFrame(generate_summaries(model, test_dataset, tokenizer, num_samples=50))

rouge_scores = rouge.compute(predictions=finetuned_summaries_df.generated_summary, references=actual_summaries.summary.iloc[:50])
bert_scores = bert_score.compute(predictions=finetuned_summaries_df.generated_summary, references=actual_summaries.summary.iloc[:50], lang="en")

print('Rouge: ', rouge_scores)
print('BERTScore Precision: ', np.mean(bert_scores['precision']))
print('BERTScore Recall: ', np.mean(bert_scores['recall']))
print('BERTScore F1: ', np.mean(bert_scores['f1']))